In [1]:
# Cell 1: Install dependencies with quiet mode
!pip install -q together chromadb rank-bm25 sentence-transformers pypdf2 pdfplumber tqdm langchain python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 6.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.3/91.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 97.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 123.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.7 MB/s eta 0

In [2]:
import os
os.environ["TOGETHER_API_KEY"] = "tgp_v1_pfYs7skW5m9MM0j55YsqOkzPR0ILywU6qY4pRCzMilU"


In [3]:
vit_questions = [
    "What is FFCS",
    "Explain the grading system",
    "How do I get admission in BTech",
    "What is the minimum duration for BCA",
    "Explain in detail relative grading",
    "What is the minimum required attendance",
    "What privileges do 9 pointers get",
    "Who are you",
    "What is the fee structure for BTech",
    "What is Program Migration in BTech",
    "What is VITEEE",
    "Who are proctors and why are students assigned a proctor in VIT",
    "What is CGPA",
    "What is SGPA",
    "How is CGPA calculated",
    "What are prerequisite courses",
    "What is the process for choosing branches, specializations, minors or honors",
    "Can students change their branch after joining",
    "What are the rules and guidelines regarding ragging on campus",
    "Are internships mandatory or optional",
    "Are there any scholarships available for students",
    "What are the disciplinary policies of VIT Chennai",
    "Is it mandatory to join or participate in club activities",
    "How do students commute within the campus",
    "Can parents visit the students on campus",
    "What is the refund policy of VIT University",
    "Are there medical facilities on campus",
    "How is the academic calendar structured",
    "Does VIT provide additional support for students preparing for higher studies",
    "What is the dress code on campus",
    "How are research opportunities provided for undergraduates",
    "Does VIT support international exchange programs",
    "Are mobile phones and/or laptops allowed in classrooms",
    "How can students contact professors after classes",
    "How are the library facilities in VIT Chennai campus",
    "What are the policies on students working part-time or freelancing during their semesters",
    "What is the difference between backlogs and arrears",
    "Is there any minimum CGPA or SGPA requirement",
    "Are students allowed to leave campus on weekdays",
    "What is the policy for medical leave or long absences",
    "How are labs evaluated",
    "What is the policy on late fee payments and penalties",
    "What all documents are required to be submitted during admission to VIT Chennai",
    "What is the policy for applying for a semester break or an academic break",
    "Are students allowed to stay/live off campus",
    "What are the rules regarding students using campus WiFi",
    "How can students appear for improvement exams",
    "For what reasons can a student be suspended or expelled from VIT",
    "Do students absolutely have to participate in fests and/or college events during academic hours",
    "What is the current policy on 'On Duty's in VIT Chennai",
    "What are the rules for visitors in hostels",
    "What are the safety and emergency protocols followed on campus",
    "How is the academic calendar structured",
    "How are faculty members selected, and what is the student-faculty ratio?",
    "Does the university provide internship assistance, and are there any industry collaborations?",
    "What is the highest and average package offered in recent years for my course?",
    "Are there any restrictions on using gadgets (laptops, phones) in classes?",
    "Are there student clubs or technical chapters (like IEEE, ACM, etc.)?",
    "Can students take additional courses or certifications (like Coursera, NPTEL) alongside their degree?",
    "Are there 24/7 study spaces or labs available for students?"
]
len(vit_questions)

60

In [4]:
# Cell 3: Import libraries and initialize components
import os
import re
import chromadb
import numpy as np
from tqdm import tqdm
import pdfplumber
from rank_bm25 import BM25Okapi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv

load_dotenv()
TOGETHER_API_KEY = os.getenv("TOGETHER_API_KEY")

In [5]:

import re

def clean_query(user_input):
    """
    Minimal cleaning: remove control chars, normalize whitespace.
    Preserves numbers, punctuation, stopwords, and original wording.
    """
    if not user_input:
        return ""
    # Remove non-printable/control characters
    cleaned = re.sub(r'[\x00-\x1f\x7f-\x9f]', '', user_input)
    # Normalize whitespace
    cleaned = re.sub(r'\s+', ' ', cleaned)
    # Strip leading/trailing whitespace
    cleaned = cleaned.strip()
    # (Optional) Lowercase if your retrieval is case-insensitive
    # cleaned = cleaned.lower()
    return cleaned


In [6]:
# Install dependencies if not already installed
# !pip install pdfplumber tqdm

import re
import pdfplumber
from tqdm import tqdm

def clean_pdf_text(text):
    if not text:
        return ""
    # Remove repeated headers/footers (customize patterns as needed)
    text = re.sub(r'Page \d+ of \d+', '', text)
    # Remove excessive whitespace
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

pdf_path = "/content/all_vitbot_data.pdf"

cleaned_pages = []
with pdfplumber.open(pdf_path) as pdf:
    for i, page in tqdm(enumerate(pdf.pages), desc="Extracting and cleaning pages", total=len(pdf.pages)):
        raw_text = page.extract_text()
        cleaned_text = clean_pdf_text(raw_text)
        cleaned_pages.append((i+1, cleaned_text))

# Show sample cleaned text from the first page
if cleaned_pages:
    print("Sample cleaned text:", cleaned_pages[0][1][:100] + "...")
else:
    print("No text extracted")


Extracting and cleaning pages: 100%|██████████| 246/246 [01:08<00:00,  3.62it/s]

Sample cleaned text: Academic Regulations Version 4.0 1. Preamble Present day student is much different from the students...


In [7]:
# # Cell 6: Text cleaning and preprocessing
# cleaned_pages = [(num, clean_text(text)) for num, text in pages]
# print("Sample cleaned text:", cleaned_pages[0][1][:100] + "...")

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len
)

chunks = []
for page_num, text in cleaned_pages:
    for chunk in splitter.split_text(text):
        chunks.append((page_num, chunk))

print(f"Created {len(chunks)} chunks")
print("Sample chunk:", chunks[3][1] + "...")

# Prepare data for indexing
chunk_texts = [chunk[1] for chunk in chunks]
metadata = [{"page_num": chunk[0]} for chunk in chunks]


Created 980 chunks
Sample chunk: a slot based timetable, students can make their own time table and student in a class may have a different timetable of his/her own. In order to make the learning more effective, the students apply the course principles by using analytical and critical thinking and thus have an opportunity to carry out challenging project(s) as part of the curriculum. Students also have the option of choosing courses from a ‘basket of courses’ within each classification. Ample options are given to choose...


In [9]:
import os
import sys

# Disable telemetry environment variable
os.environ["CHROMA_ANONYMIZED_TELEMETRY"] = "false"

# Optional: patch telemetry module to avoid errors (workaround)
sys.modules['chromadb.telemetry'] = None

from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi

# Initialize embedding model
embed_model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize ChromaDB client with telemetry disabled
chroma_client = chromadb.PersistentClient(
    path="/content/vibot_db",
    settings=Settings(anonymized_telemetry=False)
)
collection = chroma_client.get_or_create_collection(name="vibot_data")

# Add documents to ChromaDB
if chunk_texts:
    collection.add(
        documents=chunk_texts,
        metadatas=metadata,
        ids=[f"id_{i}" for i in range(len(chunk_texts))]
    )
    print(f"Indexed {len(chunk_texts)} documents in ChromaDB")
else:
    print("Warning: No documents to index!")

# Initialize BM25
if chunk_texts:
    tokenized_corpus = [doc.split(" ") for doc in chunk_texts]
    bm25 = BM25Okapi(tokenized_corpus)
    print("BM25 initialized")
else:
    print("Warning: No documents for BM25!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 32.5MiB/s]


Indexed 980 documents in ChromaDB
BM25 initialized


In [22]:
import numpy as np

def hybrid_search(query, k=15, rrf_k=60):
    try:
        cleaned_query = clean_query(query)
        if not cleaned_query:
            return []

        # --- Vector Search ---
        vector_indices = []
        try:
            query_embedding = embed_model.encode(cleaned_query).tolist()
            vector_results = collection.query(
                query_embeddings=[query_embedding],
                n_results=k
            )
            # Chroma returns ids as strings like 'id_123'
            vector_indices = [int(id.split('_')[1]) for id in vector_results['ids'][0]]
        except Exception as e:
            print(f"Vector search error: {str(e)}")
            vector_indices = []

        # --- BM25 Search ---
        bm25_indices = []
        if bm25:
            try:
                tokenized_query = cleaned_query.split(" ")
                bm25_scores = bm25.get_scores(tokenized_query)
                bm25_indices = np.argsort(bm25_scores)[::-1][:k].tolist()
            except Exception as e:
                print(f"BM25 search error: {str(e)}")
                bm25_indices = []

        # --- RRF Fusion ---
        # Create rank dictionaries
        ranks = {}
        for rank, idx in enumerate(vector_indices):
            ranks[idx] = ranks.get(idx, 0) + 1 / (rrf_k + rank + 1)
        for rank, idx in enumerate(bm25_indices):
            ranks[idx] = ranks.get(idx, 0) + 1 / (rrf_k + rank + 1)

        # Sort by RRF score
        sorted_indices = sorted(ranks.items(), key=lambda x: x[1], reverse=True)[:k]
        final_indices = [idx for idx, _ in sorted_indices]

        # Optionally, return more info (text, metadata)
        # results = [{"text": chunk_texts[idx], "meta": metadata[idx]} for idx in final_indices]
        # return results

        return final_indices

    except Exception as e:
        print(f"Hybrid search error: {str(e)}")
        return []


In [23]:
# Cell 2: Modern API setup
import together
import os

# Set API key properly (use Colab secrets or environment variable)


# Initialize client (new recommended way)
client = together.Together(api_key=os.getenv("TOGETHER_API_KEY"))

# Verify connection
try:
    models = client.models.list()
    print("API connection successful! Available models:")
    print([m.id for m in models])
except Exception as e:
    print(f"API connection failed: {str(e)}")
    print("Please verify your API key is correct and active")

API connection successful! Available models:
['Alibaba-NLP/gte-modernbert-base', 'arcee-ai/AFM-4.5B-Preview', 'arcee-ai/arcee-blitz', 'arcee-ai/caller', 'arcee-ai/coder-large', 'arcee-ai/maestro-reasoning', 'arcee-ai/virtuoso-large', 'arcee-ai/virtuoso-medium-v2', 'arcee_ai/arcee-spotlight', 'black-forest-labs/FLUX.1-canny', 'black-forest-labs/FLUX.1-depth', 'black-forest-labs/FLUX.1-dev', 'black-forest-labs/FLUX.1-dev-lora', 'black-forest-labs/FLUX.1-kontext-dev', 'black-forest-labs/FLUX.1-kontext-max', 'black-forest-labs/FLUX.1-kontext-pro', 'black-forest-labs/FLUX.1-pro', 'black-forest-labs/FLUX.1-redux', 'black-forest-labs/FLUX.1-schnell', 'black-forest-labs/FLUX.1-schnell-Free', 'black-forest-labs/FLUX.1.1-pro', 'cartesia/sonic', 'cartesia/sonic-2', 'deepseek-ai/DeepSeek-R1', 'deepseek-ai/DeepSeek-R1-0528-tput', 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B', 'deepseek-ai/DeepSeek-R1-Distill-Llama-70B-free', 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', 'deepseek-ai/DeepSeek-R1-Distil

In [27]:
def generate_response(user_input):
    try:
        cleaned_input = clean_query(user_input)
        # Retrieve more chunks to increase likelihood of capturing definitions
        relevant_indices = hybrid_search(user_input, k=12)
        # Deduplicate indices to avoid repeated chunks
        relevant_indices = list(dict.fromkeys(relevant_indices))
        context = "\n\n".join([chunk_texts[idx] for idx in relevant_indices]) if relevant_indices else ""

        # Debug: Print context passed to LLM
        print("CONTEXT PASSED TO LLM:\n", context)

        if not context:
            return "Sorry, I could not find this information."

        # More flexible prompt: allows confident inference
        prompt = (
            "You are Vibot, the official VIT college assistant. "
            "Answer the user's question as directly and clearly as possible using ONLY the context below. "
            "If the answer is present or can be directly inferred from the context, state it confidently. "
            "If not, reply: 'Sorry, I could not find this information.'\n\n"
            f"Context:\n{context}\n\n"
            f"Question: {user_input}\n"
            "Answer:"
        )

        messages = [
            {"role": "system", "content": "You are Vibot, the official VIT college assistant."},
            {"role": "user", "content": prompt}
        ]

        try:
            response = client.chat.completions.create(
                model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free",
                messages=messages,
                max_tokens=512,
                temperature=0.3
            )
            return response.choices[0].message.content.strip()
        except together.AuthenticationError:
            return "Error: Invalid API key"
        except Exception as e:
            return f"API Error: {str(e)}"
    except Exception as e:
        return f"System Error: {str(e)}"


In [28]:
# Cell 11 (Revised): Robust chat interface with additional safeguards
print("Vibot: Hi! I'm Vibot, your VIT college assistant. How can I help you today?")
print("Vibot: Type 'exit' to end our conversation.\n")

while True:
    try:
        user_input = input("You: ").strip()
        user_input = int(user_input)
        user_input = vit_questions[user_input-1]
        if not user_input:
            print("Vibot: Please enter your question.")
            continue

        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Vibot: Goodbye! Have a great day.")
            break

        # Show cleaned version for debugging
        cleaned = clean_query(user_input)
        print(f"[System] Processed query: '{cleaned}'")

        response = generate_response(user_input)
        print(f"\nVibot: {response}\n")

    except KeyboardInterrupt:
        print("\nVibot: Session ended.")
        break
    except Exception as e:
        print(f"\nVibot: Error processing request - {str(e)}")

Vibot: Hi! I'm Vibot, your VIT college assistant. How can I help you today?
Vibot: Type 'exit' to end our conversation.

You: what is FFCS
[System] Processed query: 'what is FFCS'
CONTEXT PASSED TO LLM:
 broad framework of academic course and credit requirements. FFCS allows students to decide their academic plan and permits students to alter it as they progress in time. Another milestone in implementing FFCS is moving from a fixed time table to a slot based timetable. Under this, a student will be able to choose the time he/she wants to attend a theory class/ lab. Thus, combining FFCS with a slot based timetable, students can make their own time table and student in a class may have a

Council meeting held on 27th July 2012. Minor modifications have been carried out as per the procedure outlined in Section 19 on 15th August 2012 and named as Version 2.10. FFCS Regulations 3.0, subsequent to the introduction of Curriculum for Applied Learning (CAL), was approved during the 37th Academi